# Cof Data Query 
## * Compiles adsorption data for 9 gases into 1 csv
Load all the required packages

In [2]:
from aiida import load_profile
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, Data, StructureData, KpointsData, Dict, UpfData, Int, WorkChainNode, List, CifData, CalcFunctionNode
from aiida.plugins import CalculationFactory, DataFactory
from aiida.orm import QueryBuilder
from aiida.orm import load_node
import pandas as pd
import copy

# Load our profile
load_profile()

## Query for all gases
Certain gases have have different properties requiring different queries. For example,
co2 and n2 have ranging pressures, thus we only take the constant pressures which are 0.001bar and 30bar.

## Dynamically gather all COF adsorption data
Using dictionaries we can create our Pandas dataframe by using keys as our columns and values as our entries.

In [1]:
def DataCollection(cofs, gas):
    gasPrefix, gasName = gas.split("_")
    cofData = pd.DataFrame()
        
    # For each COF
    for cof in range(len(cofs)):
        currentCof = {}
        
        # Add the current COF name
        currentCof["cof"] = cofs[cof][1]
        
        # If there's only one temperature, we just need one set of pressure values
        if type(cofs[cof][2]['temperature']) != list:
            # This set of gases don't have pressure values
            if gasPrefix == "kh":
                pass
            else:
                # co2 and n2 have massively varying pressures, thus we only add 0.001bar and 30bar
                if(gasName == "co2" or gasName == "n2"):
                    currentCof['{0}_{1}bar'.format(gasName, cofs[cof][2]['isotherm']['pressure'][0])] = cofs[cof][2]['isotherm']['loading_absolute_average'][0]
                    currentCof['{0}_{1}bar'.format(gasName, int(cofs[cof][2]['isotherm']['pressure'][-1]))] = cofs[cof][2]['isotherm']['loading_absolute_average'][-1]
                
                # In all other scenarios we want all pressure values
                else:
                    for pressure in range(len(cofs[cof][2]['isotherm']['pressure'])):
                        currentCof['{0}_{1}bar'.format(gasName, cofs[cof][2]['isotherm']['pressure'][pressure])] = cofs[cof][2]['isotherm']['loading_absolute_average'][pressure]

                # In both cases we want the valid units
                currentCof['{0}_ads_unit'.format(gasName)] = cofs[cof][2]['isotherm']['loading_absolute_unit']
        
            # Add henry coefficients for all gases with only one temperature
            currentCof['{0}_henry'.format(gasName)] = cofs[cof][2]['henry_coefficient_average']
            
        # If we have a list of temperatures, we need to add pressure values at each temperature
        else:
            
            # Iterate through each temperature pressure list and add it
            for pressureList in range(len(cofs[0][2]['temperature'])):
                for pressure in range(len(cofs[0][2]['isotherm'][pressureList]['pressure'])):
                    currentCof['{0}_{1}K_{2}bar'.format(gasName, cofs[cof][2]['temperature'][pressureList], int(cofs[cof][2]['isotherm'][pressureList]['pressure'][pressure]))] = cofs[cof][2]['isotherm'][pressureList]['loading_absolute_average'][pressure]
                
            # Add the units 
            currentCof['{0}_ads_unit'.format(gasName)] = cofs[cof][2]['isotherm'][0]['loading_absolute_unit']
            
            # Add henry coefficients for each temperature
            for temperature in range(len(cofs[0][2]['temperature'])):
                currentCof["{0}_{1}K_henry".format(gasName, cofs[cof][2]['temperature'][temperature])] = cofs[cof][2]['henry_coefficient_average'][temperature]
                
        # Henry coefficient units for all COFs
        currentCof["{0}_henry_unit".format(gasName)] = cofs[cof][2]['henry_coefficient_unit']
        
        # Append to the dataframe
        cofData = cofData.append(currentCof, ignore_index=True)
        
    # Return the dataframe
    return cofData

# Driver
Here we query for each gas to get the list of COFs. We then send that list through our DataCollection function which returns a dataframe and stores it in a dictionary.

In [5]:
# Hold all of our dataframes to be merged at the end
dataFrames = {}
# Nine gases
gases = ["isotmt_h2", "isot_o2", "isot_co2", "isot_ch4", "isot_n2",  "kh_xe", "kh_kr", "kh_h2o", "kh_h2s"]

for gas in gases:
    # Query for the specific gas
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Node, with_group='group', filters={'extras.tag4': gas, 'attributes.is_porous': True}, project='attributes')
    res = qb.all()    
    
    # Collect data for that gas
    dataFrames[gas] = DataCollection(res, gas)
    


,cof,h2_77K_1bar,h2_77K_5bar,h2_77K_25bar,h2_77K_50bar,h2_77K_75bar,h2_77K_100bar,h2_198K_1bar,h2_198K_5bar,h2_198K_25bar,...,h2_298K_5bar,h2_298K_25bar,h2_298K_50bar,h2_298K_75bar,h2_298K_100bar,h2_ads_unit,h2_77K_henry,h2_198K_henry,h2_298K_henry,h2_henry_unit
0,20610N2.cif,8.608329,18.943764,33.019479,42.110343,48.077294,52.526693,0.220603,1.091591,4.971186,...,0.515931,2.473526,4.717187,6.779165,8.921144,mol/kg,0.000133,2.244940e-06,1.024500e-06,mol/kg/Pa
1,21052N2.cif,12.492448,24.194547,40.296970,48.517777,53.710548,56.759958,0.275615,1.299559,5.850303,...,0.538124,2.491404,4.846261,7.011399,8.986940,mol/kg,0.001718,2.792180e-06,1.066640e-06,mol/kg/Pa
2,20473N2.cif,7.351015,18.898034,39.891623,56.159212,69.075600,79.111762,0.323447,1.542330,7.367576,...,0.825823,4.025714,7.847888,11.566717,15.011729,mol/kg,0.000108,3.184060e-06,1.674880e-06,mol/kg/Pa
3,15161N2.cif,11.103453,19.127537,29.268909,34.150457,36.880407,38.670559,0.182272,0.874597,3.948490,...,0.339126,1.591525,3.093829,4.469206,5.762190,mol/kg,0.001427,1.820740e-06,6.828120e-07,mol/kg/Pa
4,17061N2.cif,11.473854,15.031012,17.343098,18.156936,18.552978,18.818870,0.158416,0.736588,2.972030,...,0.214550,0.981342,1.830786,2.562661,3.230384,mol/kg,0.000622,1.601730e-06,4.399910e-07,mol/kg/Pa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,08010N2.cif,2.785746,4.578845,5.852884,6.341543,6.590769,6.709882,0.032917,0.157481,0.595548,...,0.041232,0.188842,0.348871,0.489290,0.605103,mol/kg,0.000149,3.433500e-07,8.252970e-08,mol/kg/Pa
587,20640N2.cif,9.978542,19.437045,29.850485,34.551719,37.729733,39.279700,0.190556,0.933708,4.136128,...,0.365491,1.751489,3.392045,4.861912,6.221505,mol/kg,0.000182,1.925870e-06,7.530350e-07,mol/kg/Pa
588,16056N2.cif,9.601632,16.119540,25.716533,31.354143,34.988888,37.406117,0.198080,0.947045,4.090201,...,0.381156,1.812076,3.399298,4.946513,6.367652,mol/kg,0.000285,1.963150e-06,7.689280e-07,mol/kg/Pa
589,21013N2.cif,9.128204,18.733539,32.621997,42.024845,47.935489,52.415050,0.218663,1.109665,4.897949,...,0.520836,2.483321,4.554699,6.872007,8.865178,mol/kg,0.000170,2.245880e-06,1.017690e-06,mol/kg/Pa


## Merge all dataframes
Iteratively merge each dataframe we gathered for each gas into one large csv.

In [4]:
df = copy.deepcopy(dataFrames["isotmt_h2"])
for gas in gases[1:]:
    df = df.merge(dataFrames[gas], on='cof', how='outer')

df

,cof,h2_77K_1bar,h2_77K_5bar,h2_77K_25bar,h2_77K_50bar,h2_77K_75bar,h2_77K_100bar,h2_198K_1bar,h2_198K_5bar,h2_198K_25bar,...,n2_henry,n2_henry_unit,xe_henry,xe_henry_unit,kr_henry,kr_henry_unit,h2o_henry,h2o_henry_unit,h2s_henry,h2s_henry_unit
0,20610N2.cif,8.608329,18.943764,33.019479,42.110343,48.077294,52.526693,0.220603,1.091591,4.971186,...,2.492560e-06,mol/kg/Pa,0.000022,mol/kg/Pa,0.000006,mol/kg/Pa,3.355660e-06,mol/kg/Pa,0.000021,mol/kg/Pa
1,21052N2.cif,12.492448,24.194547,40.296970,48.517777,53.710548,56.759958,0.275615,1.299559,5.850303,...,3.663020e-06,mol/kg/Pa,0.000068,mol/kg/Pa,0.000013,mol/kg/Pa,4.110900e-05,mol/kg/Pa,0.000164,mol/kg/Pa
2,20473N2.cif,7.351015,18.898034,39.891623,56.159212,69.075600,79.111762,0.323447,1.542330,7.367576,...,3.078940e-06,mol/kg/Pa,0.000021,mol/kg/Pa,0.000007,mol/kg/Pa,2.626000e-05,mol/kg/Pa,0.000019,mol/kg/Pa
3,15161N2.cif,11.103453,19.127537,29.268909,34.150457,36.880407,38.670559,0.182272,0.874597,3.948490,...,1.709490e-06,mol/kg/Pa,0.000033,mol/kg/Pa,0.000007,mol/kg/Pa,8.488530e-06,mol/kg/Pa,0.000029,mol/kg/Pa
4,17061N2.cif,11.473854,15.031012,17.343098,18.156936,18.552978,18.818870,0.158416,0.736588,2.972030,...,2.183040e-06,mol/kg/Pa,0.000144,mol/kg/Pa,0.000017,mol/kg/Pa,9.238350e-06,mol/kg/Pa,0.000122,mol/kg/Pa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,16056N2.cif,9.601632,16.119540,25.716533,31.354143,34.988888,37.406117,0.198080,0.947045,4.090201,...,2.187860e-06,mol/kg/Pa,0.000049,mol/kg/Pa,0.000009,mol/kg/Pa,6.895100e-06,mol/kg/Pa,0.000045,mol/kg/Pa
589,21013N2.cif,9.128204,18.733539,32.621997,42.024845,47.935489,52.415050,0.218663,1.109665,4.897949,...,2.538220e-06,mol/kg/Pa,0.000024,mol/kg/Pa,0.000007,mol/kg/Pa,2.143000e-04,mol/kg/Pa,0.000031,mol/kg/Pa
590,19292N2.cif,12.682302,18.935154,25.033135,27.678384,29.176354,30.008674,0.212761,0.981538,4.091099,...,2.892340e-06,mol/kg/Pa,0.000189,mol/kg/Pa,0.000022,mol/kg/Pa,1.889470e-05,mol/kg/Pa,0.000215,mol/kg/Pa
591,20542N2.cif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.535510e-06,mol/kg/Pa,0.000177,mol/kg/Pa,0.000019,mol/kg/Pa,7.180830e-06,mol/kg/Pa,NaN,NaN


## Write data to csv

In [11]:
df.dropna().to_csv('test_data.csv', index=False)